### imports


In [22]:
import os 
import folium 
from folium import plugins
import rioxarray as rxr
import earthpy as et
import earthpy.spatial as es 
from google.transit import gtfs_realtime_pb2
from google.protobuf.json_format import MessageToDict
import pandas as pd
from requests import get

### map init

In [23]:
m = folium.Map(location = [3.1319, 101.6841])

### Sample GTFS-R URL from Malaysia's Open API


In [24]:
URL = 'https://api.data.gov.my/gtfs-realtime/vehicle-position/ktmb'

### Parse the GTFS Realtime feed

In [25]:

feed = gtfs_realtime_pb2.FeedMessage()
response = get(URL)
feed.ParseFromString(response.content)

1553

### Extract and print vehicle position information

In [26]:
vehicle_positions = [MessageToDict(entity.vehicle) for entity in feed.entity]
print(f'Total vehicles: {len(vehicle_positions)}')
df = pd.json_normalize(vehicle_positions)
df

Total vehicles: 21


,timestamp,trip.tripId,position.latitude,position.longitude,position.bearing,position.speed,vehicle.id,vehicle.label
0,1700218689,weekday_2029,3.168697,101.686210,135.0,30.163033,weekday_2029,SCS 9224
1,1700218692,weekday_2007,2.903738,101.786300,0.0,2.000000,weekday_2007,SCS 9209
2,1700218692,2904,4.777111,100.862686,112.5,88.960710,2904,EMU 8329
3,1700218692,weekday_2564,3.098005,101.670654,202.5,66.011475,weekday_2564,SCS 9202
4,1700218692,weekday_2014,3.211387,101.672000,337.5,66.011475,weekday_2014,EMU 8105
5,1700218672,weekday_2114,3.031207,101.421850,225.0,26.000000,weekday_2114,SCS 9217
6,1700218672,weekday_2355,3.443763,101.638340,180.0,115.499400,weekday_2355,SCS 9225
7,1700218670,weekday_2024,2.660998,101.995544,0.0,1.000000,weekday_2024,SCS 9214
8,1700218670,9035,4.332744,101.127205,135.0,129.173000,9035,ETS 93205
9,1700218674,weekday_2347,3.211023,101.573470,315.0,71.123665,weekday_2347,SCS 9219


### mark train position

In [27]:
latitudes = df['position.latitude']
longitudes = df['position.longitude']
labels = df['vehicle.label']

for i, latitude in enumerate(latitudes):
        folium.Marker(
        location=[latitude, longitudes[i]], # coordinates for the marker (Earth Lab at CU Boulder)
        popup = labels[i], # pop-up label for the marker
    ).add_to(m)

In [28]:
m